In [1]:
#in my Windows the optimizers are located in
#/c/Users/zb0857/AppData/Local/Continuum/anaconda2/envs/Tensorflow/lib/site-packages/tensorflow/python/keras
#C:\Users\zb0857\AppData\Local\Continuum\anaconda3\envs\ML\Lib\site-packages\tensorflow\python\keras
#/c/Users/zb0857/AppData/Local/Continuum/anaconda2/envs/Tensorflow/lib/site-packages/sklearn/neural_network
#C:\Users\zb0857\AppData\Local\Continuum\anaconda3\envs\ML\Lib\site-packages\sklearn\neural_network
#rememmber to start the notebook using the environment tensorflow


from numpy.random import seed
import numpy as np
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

import tensorflow as tf
# from tensorflow.python.keras.optimizers import Fire
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import optimizers

from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import log_loss

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
from torchsummary import summary


import matplotlib.pyplot as plt
%matplotlib inline


#it seems that cateorical cross-entropy is not available in pytorch. We have to make or own
#    implementation of the loss function
#https://datascience.stackexchange.com/questions/55962/pytorch-doing-a-cross-entropy-loss-when-the-predictions-already-have-probabiliti
def categorical_cross_entropy(y_pred, y_true):
    y_pred = torch.clamp(y_pred, 1e-9, 1 - 1e-9)
    return -(y_true * torch.log(y_pred)).sum(dim=1).mean()


### CrossEntropyLoss
$\text{loss}(x, class) = -\log\left(\frac{\exp(x[class])}{\sum_j \exp(x[j])}\right)$
                       $= -x[class] + \log\left(\sum_j \exp(x[j])\right)$
                       
                       
        nll_loss(log_softmax(input, 1),


### Softmax
$\text{Softmax}(x_{i}) = \frac{\exp(x_i)}{\sum_j \exp(x_j)}$

## Preparing the input data

In [2]:
#Preparing the input data

#loading the MNIST dataset
mnist = tf.keras.datasets.mnist

#Separating into train and test (60000 train, 10000 test)
(x_train, y_train0),(x_test, y_test0) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

#flattening the images (from 28x28 to 784)
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)


#converting the otputs (labels) into one hot vectors
lb = preprocessing.LabelBinarizer()
lb.fit(np.array([0,1,2,3,4,5,6,7,8,9]))
y_train = lb.transform(y_train0)
y_test = lb.transform(y_test0)


## Defining the input parameters


In [3]:
#Defining the input parameters
BatchSize         = 5001
NeuronsLayer1     = 100
NeuronsLayer2     = 100
Epochs            = 100

## Torch by me
https://stackoverflow.com/questions/51041128/pytorch-predict-single-example

In [4]:
# Create Tensors to hold inputs and outputs

torch_X_train = torch.from_numpy(x_train)
torch_y_train = torch.from_numpy(y_train)

#the trainloader helps us deal with the batches
train = torch.utils.data.TensorDataset(torch_X_train.float(),torch_y_train.float())
train_loader = torch.utils.data.DataLoader(train, batch_size = BatchSize, shuffle = False)


In [5]:
# Defining the net

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.linear1 = nn.Linear(784,NeuronsLayer1)
        self.linear2 = nn.Linear(NeuronsLayer1,NeuronsLayer2)
        self.linear3 = nn.Linear(NeuronsLayer2,10)
    
    def forward(self,X):
        X = F.relu(self.linear1(X))
        X = F.relu(self.linear2(X))
        X = F.softmax(self.linear3(X), dim=1)
        return X



In [6]:
# Construct our model by instantiating the class defined above.
model = MLP()


# Construct our loss function and an Optimizer. 

# loss_fn = torch.nn.CrossEntropyLoss()
# loss_fn = torch.nn.NLLLoss()
# loss_fn = categorical_cross_entropy()




optimizer = torch.optim.Adam(model.parameters())


In [7]:
#Show aprameters of the network
model.parameters

<bound method Module.parameters of MLP(
  (linear1): Linear(in_features=784, out_features=100, bias=True)
  (linear2): Linear(in_features=100, out_features=100, bias=True)
  (linear3): Linear(in_features=100, out_features=10, bias=True)
)>

In [8]:
#train the network

for epoch in range(Epochs):
    loss_avg = 0.0
    N_elements = 0
    for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(X_batch)

        # Compute loss
        loss = categorical_cross_entropy(y_pred, y_batch)

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_avg += loss.item()*X_batch.size()[0]
        N_elements += X_batch.size()[0]

    # print loss after each epoch    
    print(epoch+1, loss.item(), loss_avg/N_elements)


1 1.8877276182174683 2.138044710367918
2 1.0910639762878418 1.4977989894151689
3 0.5648119449615479 0.8336560871720314
4 0.37355950474739075 0.5288091844707727
5 0.2989003360271454 0.40891184291541577
6 0.26382187008857727 0.3538558956176043
7 0.24197864532470703 0.322876588653028
8 0.22592459619045258 0.30146088517978786
9 0.21202228963375092 0.2842290604971349
10 0.19978363811969757 0.26877525778636335
11 0.18842540681362152 0.2542997420862317
12 0.178266242146492 0.2409885154530406
13 0.16919775307178497 0.22890346293002367
14 0.1609673798084259 0.2180521119400859
15 0.15359985828399658 0.20820084008723497
16 0.14685335755348206 0.19922653412297367
17 0.14064788818359375 0.19092760793417693
18 0.13503488898277283 0.18320128841996192
19 0.12974883615970612 0.17600930392742156
20 0.12490682303905487 0.1692719907581806
21 0.12005312740802765 0.16285710370093584
22 0.1158127710223198 0.15695151069872082
23 0.11181022971868515 0.1513291599806398
24 0.10798235237598419 0.14607764339596033

In [9]:
#Predicting
y_train_hat_Pytorch = model(torch_X_train.float()).detach().numpy()

print(log_loss(y_train, y_train_hat_Pytorch))
# print(categorical_cross_entropy(torch.from_numpy(y_train).float(), torch.from_numpy(y_train_hat_Pytorch)))

0.020735497776313838


In [10]:
print(y_train[0])
print(y_train[1])

[0 0 0 0 0 1 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0]


In [11]:
print(y_train_hat_Pytorch[0])
print(y_train_hat_Pytorch[1])

[2.1891752e-13 6.0890420e-14 1.8599314e-10 2.2093031e-04 1.9059933e-25
 9.9977905e-01 2.4493364e-15 7.8256352e-13 7.8601859e-14 1.5986865e-13]
[9.9999964e-01 8.2255425e-11 3.4687238e-07 7.4164890e-11 3.4500973e-16
 7.4679520e-13 2.5321070e-08 2.9021292e-11 1.1283031e-13 4.7710031e-08]


In [12]:
#Show aprameters of the network

summary(model, (1, 784))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 100]          78,500
            Linear-2               [-1, 1, 100]          10,100
            Linear-3                [-1, 1, 10]           1,010
Total params: 89,610
Trainable params: 89,610
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.34
Estimated Total Size (MB): 0.35
----------------------------------------------------------------
